## Atividades Bases da Computação III

Curso: Sistemas de Informação (3º Semestre)

------------

## Atividade 3

Essa atividade visa conectarmos os conteúdos vistos até esse momento.

Peço que redefinam a classe Fila começando pelo seu único atributo `data`, em vez de usar um vetor do Python, usar uma instância de Lista. 
Para isso, devem ser implementados novos métodos na classe Lista, descubra quais são. 
Fazer todas as adaptações necessárias na classe Fila (seu atributo e métodos).

Quando estiver com a classe Fila redefinida, implementar uma solução para o 
seguinte problema: 
- Dado um arquivo de texto de nome `inscritos.txt` contendo uma lista de nomes (string) e a categoria de atendimento (normal: 'n', ou prioritário: 'p'), com o separador vírgula em cada linha (Ex: Fulano de Tal, 'p'); implemente um código em Python que leia esse arquivo e gere uma lista de atendimentos a serem realizados em um determinado órgão público.  
- A ordem de atendimento deve ser: a cada cinco prioritários que saem da fila, três normais também são atendidos e assim sucessivamente. Ou 
seja, seguindo a lógica p, p, p, p, p, n, n, n, p, p, p, p, p, n, n, n... 
- Dentro de cada categoria a ordem é a mesma lida no arquivo.  
- Os atendimentos são realizados de meia em meia hora, começando o primeiro às 11h00 e terminando o último às 15h30, dentro de um mesmo 
dia, ou seja, o horário de atendimento deve ser impresso, junto com o nome e a categoria. 
- Caso a lista tenha mais nomes que a quantidade de atendimentos possíveis dentro de um dia de expediente, a solução deve imprimir o 
atendimento para quantos dias que forem necessários, colocando um texto "1º dia", "2º dia", ... e assim por diante, antes de listar os 
atendimentos de cada dia. 
- O resultado do código deve ser salvo em um arquivo de nome `ordemAtendimento.txt`. Esse arquivo deve ter a lista com a ordem de 
atendimento com a informação do dia, do horário, do nome da pessoa e da sua categoria, em cada linha. 

### Definição das Classes

In [13]:
class No:
    def __init__(self, conteudo, proximo=None):
        self.conteudo = conteudo
        self.proximo = proximo

In [14]:
class Lista:
    def __init__(self):
        self.cabeca = No(-1)
        self.fim = self.cabeca

    def vazia(self):
        return self.cabeca == self.fim

    def insereNoFim(self, x):
        novo_no = No(x)
        self.fim.proximo = novo_no
        self.fim = novo_no

    def removeInicio(self):
        if self.vazia():
            print("A lista está vazia.")
            return None
        primeiro = self.cabeca.proximo.conteudo
        self.cabeca.proximo = self.cabeca.proximo.proximo
        if self.cabeca.proximo is None:
            self.fim = self.cabeca
        return primeiro

    def imprime(self):
        atual = self.cabeca.proximo
        while atual is not None:
            print(atual.conteudo, end=" -> ")
            atual = atual.proximo
        print("None")

In [15]:
class Fila:
    def __init__(self):
        self.data = Lista()

    def enfileira(self, x):
        self.data.insereNoFim(x)

    def desenfileira(self):
        return self.data.removeInicio()

    def vazia(self):
        return self.data.vazia()

    def imprime(self):
        self.data.imprime()

### Função para processar a ordem de atendimento

In [16]:
def processaAtendimentos(nome_arquivo_entrada, nome_arquivo_saida):
    # Lendo o arquivo de inscritos
    with open(nome_arquivo_entrada, "r") as f:
        linhas = f.readlines()

    fila_normal = Fila()
    fila_prioritario = Fila()

    for linha in linhas:
        nome, categoria = linha.strip().split(", ")
        if categoria == 'n':
            fila_normal.enfileira(nome)
        elif categoria == 'p':
            fila_prioritario.enfileira(nome)

    # Gerando o cronograma de atendimentos
    horario_inicio = 11 * 60  # Horário inicial em minutos (11h00)
    intervalo = 30
    horario_maximo = 15 * 60 + 30  # Horário final em minutos (15h30)
    atendimentos_por_dia = (horario_maximo - horario_inicio) // intervalo
    ordem_atendimento = []
    dia = 1

    while not fila_normal.vazia() or not fila_prioritario.vazia():
        ordem_atendimento.append(f"Dia {dia}:")
        horario_atual = horario_inicio
        count_atendimentos = 0

        while horario_atual < horario_maximo and (not fila_normal.vazia() or not fila_prioritario.vazia()):
            for _ in range(5):  # Atender 5 prioritários
                if not fila_prioritario.vazia() and count_atendimentos < atendimentos_por_dia:
                    nome = fila_prioritario.desenfileira()
                    horario_formatado = f"{horario_atual // 60:02}:{horario_atual % 60:02}"
                    ordem_atendimento.append(f"{horario_formatado} - {nome} (prioritario)")
                    horario_atual += intervalo
                    count_atendimentos += 1

            for _ in range(3):  # Atender 3 normais
                if not fila_normal.vazia() and count_atendimentos < atendimentos_por_dia:
                    nome = fila_normal.desenfileira()
                    horario_formatado = f"{horario_atual // 60:02}:{horario_atual % 60:02}"
                    ordem_atendimento.append(f"{horario_formatado} - {nome} (normal)")
                    horario_atual += intervalo
                    count_atendimentos += 1

        dia += 1  # Passa para o próximo dia

    # Salvando o cronograma no arquivo de saída
    with open(nome_arquivo_saida, "w") as f:
        for linha in ordem_atendimento:
            f.write(linha + "\n")

# Testando a solução
processaAtendimentos("inscritos.txt", "ordemAtendimento.txt")

> Favor checar o arquivo de output `ordemAtendimento.txt`